# Homework 1

_**[Energy Systems Optimization](https://github.com/nspatank/energy_system_optimization)**_

_by Neha Patankar (last updated: September 23, 2023)_

This Notebook will walk you through defining a simple transport flow model and then ask you to interact with the solutions and modify to model to add additional constraints...

## Setting up the model

### Load packages

In [1]:
using JuMP
using HiGHS
using DataFrames
using CSV

### Define sets

We will define two sets, both as arrays of strings

***Production plants, $P$***

In [2]:
P=["trenton", "newark"] # production plants

2-element Vector{String}:
 "trenton"
 "newark"

***Markets for products, $M$***

In [3]:
M=["newyork", "princeton", "philadelphia"] # markets for products

3-element Vector{String}:
 "newyork"
 "princeton"
 "philadelphia"

Note that sets can also be defined over intervals (as in `i=1:10`) or numerical vectors (as in `x=[2, 4, 5, 11]`) 

### Define parameters

We'll make use of the defined sets as indexes for our parameters...

***Plant production capacities***

In [4]:
plants = DataFrame(plant=P, capacity=[350,650])

Row,plant,capacity
,String,Int64
1,trenton,350
2,newark,650


***Demand for products***

Stored in a [DataFrame](https://juliadata.github.io/DataFrames.jl/stable/)

In [5]:
markets = DataFrame(
    market=M, 
    demand=[325, 300, 275]
)

Row,market,demand
,String,Int64
1,newyork,325
2,princeton,300
3,philadelphia,275


A few different ways to index into our DataFrames to access parameters (all of the below are equivalent)

In [6]:
plants[plants.plant.=="newark",:capacity] # option 1

1-element Vector{Int64}:
 650

In [7]:
plants[plants.plant.=="newark",:].capacity # option 2

1-element Vector{Int64}:
 650

In [8]:
plants.capacity[plants.plant.=="newark"] # option 3

1-element Vector{Int64}:
 650

In [9]:
plants[:,:capacity][plants.plant.=="newark"] # option 4

1-element Vector{Int64}:
 650

Note that DataFrame indexing returns an Array by default, in this case, a 1-element Array of type Int64 (64-bit integer), as indicated by `Array{Int64,1}` above. 

To access the single Int64 value, append `[1]` to any of the above to reference the first (and only) element in this array. 

In [10]:
plants.capacity[plants.plant.=="newark"][1]

650

In [11]:
typeof(plants.capacity[plants.plant.=="newark"][1])

Int64

In [12]:
typeof(plants.capacity[plants.plant.=="newark"])

Vector{Int64} (alias for Array{Int64, 1})

***Distance from plants to markets***

Stored in a JuMP [DenseAxisArray](https://jump.dev/JuMP.jl/v0.19/containers/) with data array and symbolic references across each of our sets (plants and markets), converted to Symbols for referencing

In [13]:
# two dimensional symbolic DenseAxisArray, with from/to distance pairs
distances = JuMP.Containers.DenseAxisArray(
    [2.5 0.5 1.5;
     0.5 1.5 3.5],
    Symbol.(P),
    Symbol.(M),
)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:trenton, :newark]
    Dimension 2, [:newyork, :princeton, :philadelphia]
And data, a 2×3 Matrix{Float64}:
 2.5  0.5  1.5
 0.5  1.5  3.5

A couple example references to our DenseAxisArray to access parameters...

In [14]:
distances[:trenton, :newyork] #example of distance references

2.5

In [15]:
distances[:newark, :newyork] #example of distance references

0.5

In [16]:
distances[Symbol(P[2]),Symbol(M[1])] # another way to find distance from newark to trenton

0.5

In [17]:
distances[Symbol("newark"), Symbol("newyork")] # and a third...

0.5

***Costs of transport***

In [18]:
freight_cost = 90 # Cost of freight shipment per unit of distance

90

### Create model
(and specify the Clp solver)

In [19]:
transport = Model(HiGHS.Optimizer);

### Define variables

***Quantities of product to transport from plant $p \in P$ to market $m \in M$***

In [20]:
@variable(transport, X[P,M] >= 0)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["trenton", "newark"]
    Dimension 2, ["newyork", "princeton", "philadelphia"]
And data, a 2×3 Matrix{VariableRef}:
 X[trenton,newyork]  X[trenton,princeton]  X[trenton,philadelphia]
 X[newark,newyork]   X[newark,princeton]   X[newark,philadelphia]

Example reference to single quantity decision variable, the quantity shipped from Newark to Philadelphia:

In [21]:
X["newark","philadelphia"]

X[newark,philadelphia]

### Define constraints

***Supply capacity constraint***

In [22]:
@constraint(transport, cSupply[p in P], 
    sum(X[p,m] for m in M) 
    <= plants.capacity[plants.plant.==p][1])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["trenton", "newark"]
And data, a 2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 cSupply[trenton] : X[trenton,newyork] + X[trenton,princeton] + X[trenton,philadelphia] <= 350
 cSupply[newark] : X[newark,newyork] + X[newark,princeton] + X[newark,philadelphia] <= 650

***Demand balance constraint***

Ensure all demand is satisfied at each market

In [23]:
@constraint(transport, cDemand[m in M], 
    sum(X[p,m] for p in P) 
    >= markets.demand[markets.market.==m][1])

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, ["newyork", "princeton", "philadelphia"]
And data, a 3-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.GreaterThan{Float64}}, ScalarShape}}:
 cDemand[newyork] : X[trenton,newyork] + X[newark,newyork] >= 325
 cDemand[princeton] : X[trenton,princeton] + X[newark,princeton] >= 300
 cDemand[philadelphia] : X[trenton,philadelphia] + X[newark,philadelphia] >= 275

### Define objective function

Minimize total cost of transport to satisfy all demand.

First we'll define an expression for total cost of shipments...

In [24]:
@expression(transport, eCost, 
    sum(freight_cost*distances[Symbol(p),Symbol(m)]*X[p,m] 
        for p in P, m in M)
    )

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia]

Now we'll minimize this total cost

In [25]:
@objective(transport, Min, eCost)

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia]

## Interact with the model

**(a)** Now let's solve the model. In the blank cell below, enter the command for JuMP to solve a model and run the cell

In [26]:
optimize!(transport)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [4e+01, 3e+02]
  Bound  [0e+00, 0e+00]
  RHS    [3e+02, 6e+02]
Presolving model
5 rows, 6 cols, 12 nonzeros  0s
5 rows, 6 cols, 12 nonzeros  0s
Presolve : Reductions: rows 5(-0); columns 6(-0); elements 12(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(900) 0s
          4     8.5500000000e+04 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 4
Objective value     :  8.5500000000e+04
HiGHS run time      :          0.00


**(b)** You've got a solution. Now query the objective function in the empty cell below and save it to a variable (name of your choice)

In [27]:
total_cost = objective_value(transport)

85500.0

In [28]:
summary = solution_summary(transport, verbose=true)

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 8.55000e+04
  Objective bound    : 0.00000e+00
  Relative gap       : Inf
  Dual objective value : 8.55000e+04
  Primal solution :
    X[newark,newyork] : 3.25000e+02
    X[newark,philadelphia] : 0.00000e+00
    X[newark,princeton] : 2.25000e+02
    X[trenton,newyork] : 0.00000e+00
    X[trenton,philadelphia] : 2.75000e+02
    X[trenton,princeton] : 7.50000e+01
  Dual solution :
    cDemand[newyork] : 4.50000e+01
    cDemand[philadelphia] : 2.25000e+02
    cDemand[princeton] : 1.35000e+02
    cSupply[newark] : 0.00000e+00
    cSupply[trenton] : -9.00000e+01

* Work counters
  Solve time (sec)   : 0.00000e+00
  Simplex iterations : 4
  Barrier iterations : 0
  Node count         : -1


**(c)** Now query and save the optimal solution for X (the decisions about shipment quantities from plant to market) to an Array or DataFrame

In [29]:

optimal_solution = DataFrame([(p, m, value(X[p, m])) for p in P for m in M if value(X[p, m]) > 0], [:Plant, :Market, :Quantity])


Row,Plant,Market,Quantity
,String,String,Float64
1,trenton,princeton,75.0
2,trenton,philadelphia,275.0
3,newark,newyork,325.0
4,newark,princeton,225.0


**(d)** Please interpret your results by writing an explanation in the markdown cell below. 

Which facility or facilities supplies the most demand in New York? Does this result make sense? Why?

Which facility or facilities supplies the most demand in Philadelphia? Does this result make sense? Why?

Which facility or facilities supplies the demand in Princeton? Does this result make sense? Why?

Newark supplies all the demand in New York. This makes sense because Newark is adjacent to New York and has enough capacity to supply all the demand in New York.
Trenton supplies all the demand in Philadelphia. Trenton is close to Philadelphia and has enough capacity to cover Philadelphia, so this makes sense.

Both Trenton and Newark supplies electricity to Princeton. However, despite being closer to Princeton, Trenton supplies less than Newark. This is because Philadelphia also needs supply, and Trenton is closer to Philadelphia than Newark.  i.e. the difference in distance is more significant in case of Philadelphia. So Trenton prioritizes Philadelphia over Princeton. Since Trenton's capacity is lower than Newark, it cannot satisfy the demand of both the cities, and some of that is fulfilled by Newark.

**(e)** A new market in New Brunswick appears, with a demand for 50 units. It is located 1.0 units away from both plants. Add this market to the model and solve again.

In [30]:
P=["trenton", "newark"] # production plants
M_revised = ["newyork", "princeton", "philadelphia", "newbrunswick"] # markets for products
plants = DataFrame(plant=P, capacity=[350,650])
markets_revised = DataFrame(
    market=M_revised, 
    demand=[325, 300, 275, 50]
)
distances = JuMP.Containers.DenseAxisArray(
    [2.5 0.5 1.5 1.0;
     0.5 1.5 3.5 1.0],
    Symbol.(P),
    Symbol.(M_revised),
)
freight_cost = 90 # Cost of freight shipment per unit of distance
transport_revised = Model(HiGHS.Optimizer);
@variable(transport_revised, Y[P,M_revised] >= 0) # changed variable name to Y
@constraint(transport_revised, cSupply[p in P], 
    sum(Y[p,m] for m in M_revised) 
    <= plants.capacity[plants.plant.==p][1])
@constraint(transport_revised, cDemand[m in M_revised],
    sum(Y[p,m] for p in P) 
    >= markets_revised.demand[markets_revised.market.==m][1])
@expression(transport_revised, eCost,
    sum(freight_cost*distances[Symbol(p),Symbol(m)]*Y[p,m] 
        for p in P, m in M_revised)
    )
@objective(transport_revised, Min, eCost)
optimize!(transport_revised)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [4e+01, 3e+02]
  Bound  [0e+00, 0e+00]
  RHS    [5e+01, 6e+02]
Presolving model
6 rows, 8 cols, 16 nonzeros  0s
6 rows, 8 cols, 16 nonzeros  0s
Presolve : Reductions: rows 6(-0); columns 8(-0); elements 16(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 4(950) 0s
          5     9.0000000000e+04 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  9.0000000000e+04
HiGHS run time      :          0.00


**(f)** What is new optimal solution? 

In [31]:
objective_value(transport_revised)

90000.0

In [32]:
optimal_solution_revised = DataFrame([(p, m, value(Y[p, m])) for p in P for m in M_revised if value(Y[p, m]) > 0], [:Plant, :Market, :Quantity])

Row,Plant,Market,Quantity
,String,String,Float64
1,trenton,princeton,75.0
2,trenton,philadelphia,275.0
3,newark,newyork,325.0
4,newark,princeton,225.0
5,newark,newbrunswick,50.0


In [33]:
solution_summary(transport_revised, verbose=true)

* Solver : HiGHS

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "kHighsModelStatusOptimal"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 9.00000e+04
  Objective bound    : 0.00000e+00
  Relative gap       : Inf
  Dual objective value : 9.00000e+04
  Primal solution :
    Y[newark,newbrunswick] : 5.00000e+01
    Y[newark,newyork] : 3.25000e+02
    Y[newark,philadelphia] : 0.00000e+00
    Y[newark,princeton] : 2.25000e+02
    Y[trenton,newbrunswick] : 0.00000e+00
    Y[trenton,newyork] : 0.00000e+00
    Y[trenton,philadelphia] : 2.75000e+02
    Y[trenton,princeton] : 7.50000e+01
  Dual solution :
    cDemand[newbrunswick] : 9.00000e+01
    cDemand[newyork] : 4.50000e+01
    cDemand[philadelphia] : 2.25000e+02
    cDemand[princeton] : 1.35000e+02
    cSupply[newark] : 0.00000e+00
    cSupply[trenton] : -9.00000e+01

* Work counters
  Solve time (sec)   : 7

**(j)** Interpret this result in the markdown cell below. Which facility or facilities supplies the demand in New Brunswick? Does this result make sense? Why?

Newark supplies New Brunswick despite both plants being equidistant from New Brunswick. This is because Newark has more capacity than Trenton and all of Trenton's capacity is better served by supplying Philadelphia, to which it's closer. So Newark supplies New Brunswick.